# Dog vs Cat Detection
Training a neural network to be able to differentiate between images of dogs and cats.
Doesn't go through images in any order.

## Preprocessing Data

In [ ]:
# importing dependencies
import numpy as np
from numpy import asarray
import cv2
import os
from os import listdir

In [ ]:
# path for data, numpy arrays
DATA_PATH = os.path.join('./Data') 

IMG_PATH = './images/train'

length = 30

# Actions that we try to detect
pets = np.array(['cats', 'dogs'])

# create directories
for pet in pets: 
    try: 
        os.makedirs(os.path.join(DATA_PATH, pet))
    except:
        pass

In [ ]:
def createArray(filePath):
    img = cv2.imread()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return np.array(img)  

In [ ]:
def preprocess_image(image_array, target_size):
    image_resized = cv2.resize(image_array, target_size)
    image_normalized = image_resized / 255.0
    return image_normalized

In [ ]:
# convert training images to numpy arrays and store in data folder
target_size(224, 224)
for pet in pets: 
    index = 0
    for subject in os.listdir(f'{IMG_PATH}/{pet}'):
        if subject == '.DS_Store':
            continue
        else:
            arr = createArray(f'{IMG_PATH}/{pet}/{subject}')
            np_arr = preprocess_image(arr, target_size)
            np.save(f'{DATA_PATH}/{pet}/{pet}{index}.npy', np_arr)
            index += 1

## Creating Features and Labels

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(pets)}

In [ ]:
label_map

In [ ]:
features, labels = [], []
for pet in pets:
    window = []
    for subject in os.listdir(os.path.join(DATA_PATH, pet)):
        n = np.load(f'{DATA_PATH}/{pet}/{subject}')
        window.append(n)
    features.append(window)
    labels.append(label_map[pet])

In [133]:
np.array(features, dtype=object).shape

(2, 12500)

In [134]:
np.array(labels).shape

(2,)

In [135]:
X = np.array(features)

/var/folders/xn/2lpjgth50sn7rh3dx6bz20nm0000gn/T/ipykernel_5671/3919513189.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(features)


In [136]:
y = to_categorical(labels).astype(int)

In [137]:
# 80:20 ratio between training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

## Building and Training the Neural Network

In [138]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard

In [139]:
# view data on the model as it's being trained
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [140]:
model = Sequential([
    Dense(units=64, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=1, activation='softmax')
])

In [141]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])

In [142]:
model.fit(X_train, y_train, epochs=10, callbacks=[tb_callback])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
model.summary()

## Making Predictions